### TF_IDF Vectorization

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv('AI Based Book Recommendation System/dataset/final_data.csv')

# Preprocess the data
df.fillna('', inplace=True)
df['Desc'] = df['Desc'].str.lower()
df['Title'] = df['Title'].str.lower()

# Create a tf-idf vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Desc'] + ' ' + df['Title'] + ' ' + df['Genre'])
tfidf_matrix = tfidf_matrix.astype(np.float32)

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Save the cosine similarity matrix
np.save('AI Based Book Recommendation System/model/cosine_sim.npy', cosine_sim)

In [6]:
def recommend_books_cosine(book_title, final_data, cosine_sim):

    # Check if the final_data DataFrame is empty
    if not final_data.empty:
        # Get the index of the book title
        idx = final_data[final_data['Title'] == book_title].index
        print(f"idx: {idx}")
        if len(idx) > 0:
            idx = idx[0]
            sim_scores = list(enumerate(cosine_sim[idx]))
            print(f"sim_scores: {sim_scores}")
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]
            print(f"sim_scores top 10: {sim_scores}")
            book_indices = [i[0] for i in sim_scores]
            print(f"book_indices: {book_indices}")
            # return book title with image url and author
            return final_data[['Title', 'Image', 'Author']].iloc[book_indices]
        else:
            return "Book not found"
    else:
        return "No data available"

# Example usage
df = pd.read_csv('AI Based Book Recommendation System/dataset/final_data.csv')
cosine_sim = np.load('AI Based Book Recommendation System/model/cosine_sim.npy')
recommend_books_cosine('Python for Data Analysis', df, cosine_sim)


idx: Int64Index([17273], dtype='int64')
sim_scores: [(0, 0.052499466), (1, 0.006910086), (2, 0.014754174), (3, 0.1083718), (4, 0.023390446), (5, 0.0), (6, 0.0), (7, 0.035820235), (8, 0.02284322), (9, 0.0403712), (10, 0.011541335), (11, 0.010319863), (12, 0.011790952), (13, 0.0), (14, 0.0770881), (15, 0.01734209), (16, 0.024195436), (17, 0.011981061), (18, 0.02952066), (19, 0.002775732), (20, 0.015381871), (21, 0.037851065), (22, 0.01493819), (23, 0.016667154), (24, 0.016730107), (25, 0.0), (26, 0.08717003), (27, 0.024586525), (28, 0.0059973733), (29, 0.016020594), (30, 0.012602329), (31, 0.011102149), (32, 0.0), (33, 0.020326976), (34, 0.04692241), (35, 0.012366842), (36, 0.061682064), (37, 0.002117056), (38, 0.015397036), (39, 0.003181843), (40, 0.0), (41, 0.0040365355), (42, 0.013023352), (43, 0.06450635), (44, 0.042030945), (45, 0.012059403), (46, 0.008196373), (47, 0.016456507), (48, 0.01817279), (49, 0.009498764), (50, 0.0033322682), (51, 0.028735735), (52, 0.03082693), (53, 0.013

,Title,Image,Author
7587,Think Stats,https://i.gr-assets.com/images/S/compressed.ph...,Allen B. Downey
21710,Interactive Data Visualization for the Web,https://i.gr-assets.com/images/S/compressed.ph...,Scott Murray
4176,Visualize This: The FlowingData Guide to Desig...,https://i.gr-assets.com/images/S/compressed.ph...,Nathan Yau
17397,The Elements of Statistical Learning: Data Min...,https://i.gr-assets.com/images/S/compressed.ph...,"Trevor Hastie,Robert Tibshirani,Jerome Friedman"
11727,Seven Databases in Seven Weeks: A Guide to Mod...,https://i.gr-assets.com/images/S/compressed.ph...,"Eric Redmond,Jim R. Wilson"
14950,NoSQL Distilled: A Brief Guide to the Emerging...,https://i.gr-assets.com/images/S/compressed.ph...,"Pramod J. Sadalage,Martin Fowler"
16922,Think Python,https://i.gr-assets.com/images/S/compressed.ph...,Allen B. Downey
20103,Big Data: A Revolution That Will Transform How...,https://i.gr-assets.com/images/S/compressed.ph...,"Viktor Mayer-SchÃ¶nberger,Kenneth Cukier"
20745,Predictive Analytics: The Power to Predict Who...,https://i.gr-assets.com/images/S/compressed.ph...,Eric Siegel
23440,Computer Networks,https://i.gr-assets.com/images/S/compressed.ph...,Andrew S. Tanenbaum


### NCF Model

#### Pre-Processing dataset 

In [21]:
# Assume 1000 users and assign random ratings between 1 and 5 to each book by users
num_users = 1000
num_ratings = len(df)

# Generate random user IDs
user_ids = np.random.randint(1, num_users + 1, num_ratings)

# Generate random ratings
ratings = np.random.randint(1, 6, num_ratings)

# Add user IDs and ratings to the dataset
df['user_id'] = user_ids
df['rating'] = ratings

# Encode the user IDs and ISBNs
df['user_id'] = df['user_id'].astype(
    'category').cat.codes.values
df['ISBN'] = df['ISBN'].astype('category').cat.codes.values

# Save the final_data to a CSV file
df.to_csv("AI Based Book Recommendation System/dataset/final_data_with_ratings.csv", index=False)


#### Model building

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam

# Prepare the data for NCF
df = pd.read_csv('AI Based Book Recommendation System/dataset/final_data_with_ratings.csv')
df['user_id'] = df['user_id'].astype(
    'category').cat.codes.values
df['ISBN'] = df['ISBN'].astype('category').cat.codes.values

# Select necessary columns for NCF
user_item_data = df[['user_id', 'ISBN', 'Rating']]

# Split the data into training and testing sets
train, test = train_test_split(user_item_data, test_size=0.2, random_state=42)

# Define the NCF model
num_users = user_item_data['user_id'].nunique()
num_items = user_item_data['ISBN'].nunique()

# User input and embedding
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(
    input_dim=num_users, output_dim=50, name='user_embedding')(user_input)
user_vec = Flatten(name='user_flatten')(user_embedding)

# Item input and embedding
item_input = Input(shape=(1,), name='item_input')
item_embedding = Embedding(
    input_dim=num_items, output_dim=50, name='item_embedding')(item_input)
item_vec = Flatten(name='item_flatten')(item_embedding)

# Concatenate the embeddings
concat = Concatenate(name='concat')([user_vec, item_vec])
dense = Dense(128, activation='relu', name='dense1')(concat)
dropout = Dropout(0.3, name='dropout')(dense)
output = Dense(1, activation='linear', name='output')(dropout)

# Compile the model
model = Model([user_input, item_input], output)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error', metrics=['mae']) # mae = Mean Absolute Error

# Train the model
history = model.fit([train['user_id'], train['ISBN']], train['Rating'],
                    validation_data=(
                        [test['user_id'], test['ISBN']], test['Rating']),
                    epochs=10, batch_size=64, verbose=1)

# Save the model
model.save('AI Based Book Recommendation System/model/ncf_model.h5')

# Evaluate the model
loss, mae = model.evaluate(
    [test['user_id'], test['ISBN']], test['Rating'], verbose=0)
print(f'NCF Model Test MAE: {mae}')


Epoch 1/10


315/315 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - loss: 7.6892 - mae: 2.3463 - val_loss: 0.1323 - val_mae: 0.2893
Epoch 2/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 12s 37ms/step - loss: 0.2094 - mae: 0.3648 - val_loss: 0.1193 - val_mae: 0.2759
Epoch 3/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - loss: 0.1762 - mae: 0.3339 - val_loss: 0.1149 - val_mae: 0.2691
Epoch 4/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - loss: 0.1495 - mae: 0.3072 - val_loss: 0.1118 - val_mae: 0.2682
Epoch 5/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 0.1345 - mae: 0.2916 - val_loss: 0.1073 - val_mae: 0.2626
Epoch 6/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - loss: 0.1233 - mae: 0.2791 - val_loss: 0.0967 - val_mae: 0.2486
Epoch 7/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 0.1124 - mae: 0.2675 - val_loss: 0.0944 - val_mae: 0.2433
Epoch 8/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.1077 - mae: 0.2616 - val_loss: 0.0935 - val_mae: 0.2431
Epoch 9/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step 

NCF Model Test MAE: 0.23785340785980225


### CNN Model

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Prepare text data
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['Desc'])
sequences = tokenizer.texts_to_sequences(df['Desc'])
data = pad_sequences(sequences, maxlen=max_len)

# Prepare labels
labels = df['Rating'].values

# Split data into training and testing sets
train_size = int(0.8 * len(data))
x_train, x_test = data[:train_size], data[train_size:]
y_train, y_test = labels[:train_size], labels[train_size:]

# Define the CNN model
model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate the model
mae = model.evaluate(x_test, y_test)[1]
print(f'CNN Model Test MAE: {mae}')

# Save the model
model.save('AI Based Book Recommendation System/model/cnn_model.h5')


Epoch 1/10


f:\Software\Anaconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


252/252 ━━━━━━━━━━━━━━━━━━━━ 39s 142ms/step - loss: 2.3839 - mae: 1.0933 - val_loss: 0.1515 - val_mae: 0.3146
Epoch 2/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 26s 104ms/step - loss: 0.3968 - mae: 0.5007 - val_loss: 0.0850 - val_mae: 0.2278
Epoch 3/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 43s 112ms/step - loss: 0.3838 - mae: 0.4948 - val_loss: 0.0861 - val_mae: 0.2325
Epoch 4/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 27s 107ms/step - loss: 0.3546 - mae: 0.4771 - val_loss: 0.1303 - val_mae: 0.3008
Epoch 5/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 54s 158ms/step - loss: 0.3410 - mae: 0.4666 - val_loss: 0.1373 - val_mae: 0.3090
Epoch 6/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 40s 155ms/step - loss: 0.3224 - mae: 0.4531 - val_loss: 0.1597 - val_mae: 0.3351
Epoch 7/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 38s 150ms/step - loss: 0.3172 - mae: 0.4501 - val_loss: 0.0855 - val_mae: 0.2334
Epoch 8/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 40s 158ms/step - loss: 0.3010 - mae: 0.4376 - val_loss: 0.0966 - val_mae: 0.2507
Epoch 9/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 38s

CNN Model Test MAE: 0.23906251788139343


### Hybrid Model

In [9]:
## Baseline Recommendation System
def recommend_books_cosine(book_title, final_data, cosine_sim):

    # Check if the final_data DataFrame is empty
    if not final_data.empty:
        # Get the index of the book title
        idx = final_data[final_data['Title'] == book_title].index
        print(f"idx: {idx}")
        if len(idx) > 0:
            idx = idx[0]
            sim_scores = list(enumerate(cosine_sim[idx]))
            print(f"sim_scores: {sim_scores}")
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]
            print(f"sim_scores top 10: {sim_scores}")
            book_indices = [i[0] for i in sim_scores]
            print(f"book_indices: {book_indices}")
            # return book title with image url and author
            return final_data[['Title', 'Image', 'Author']].iloc[book_indices]
        else:
            return "Book not found"
    else:
        return "No data available"

In [26]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch, pandas as pd, tensorflow as tf
import numpy as np
def hybrid_recommendation(book_title, df, cosine_sim, ncf_model, cnn_model, tokenizer, max_len=200):
    # Cosine similarity recommendations
    cosine_recs = recommend_books_cosine(book_title, df, cosine_sim)
    
    # NCF recommendations using the trained model
    user_id = df['user_id'].iloc[0]  # Replace with the actual user ID
    book_ids = [df[df['Title'] == title]['ISBN'].values[0] for title in cosine_recs['Title']]
    ncf_recs = []
    for book_id in book_ids:
        user_tensor = torch.tensor([int(user_id)], dtype=torch.long)
        book_tensor = torch.tensor([int(book_id)], dtype=torch.long)
        rating_pred = ncf_model([user_tensor, book_tensor])[0].numpy()
        # rating_pred = tf.reshape(ncf_model([user_tensor, book_tensor]), [-1]).numpy()
        ncf_recs.append((book_id, rating_pred))
    
    ncf_recs = sorted(ncf_recs, key=lambda x: x[1], reverse=True)
        
    # CNN recommendations
    cnn_recs = []
    for book_id in book_ids:
        book_desc = df[df['ISBN'] == book_id]['Desc'].values[0]
        book_seq = tokenizer.texts_to_sequences([book_desc])
        book_pad = pad_sequences(book_seq, maxlen=max_len)
        rating_pred = cnn_model.predict(book_pad).item()
        cnn_recs.append((book_id, rating_pred))
    
    cnn_recs = sorted(cnn_recs, key=lambda x: x[1], reverse=True)
    
    # Combine and rank recommendations
    combined_recs = list(set( [book for book, _ in cnn_recs]))
    final_recs = [(df[df['ISBN'] == book_id]['Title'].values[0], 
                   df[df['ISBN'] == book_id]['Author'].values[0], 
                   df[df['ISBN'] == book_id]['Genre'].values[0]) 
                  for book_id in combined_recs]
    
    return [final_recs[i][0] for i in range(len(final_recs))]

# Usage
cosine_sim = np.load('AI Based Book Recommendation System/model/cosine_sim.npy')
ncf_model = load_model('AI Based Book Recommendation System/model/ncf_model.h5')
cnn_model = load_model("AI Based Book Recommendation System/model/cnn_model.h5")
df = pd.read_csv("AI Based Book Recommendation System/dataset/final_data_with_ratings.csv")
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['Desc'])
hybrid_recommendations = hybrid_recommendation('Python for Data Analysis', df, cosine_sim, ncf_model, cnn_model, tokenizer)
print(hybrid_recommendations)


idx: Int64Index([17273], dtype='int64')
sim_scores: [(0, 0.052499466), (1, 0.006910086), (2, 0.014754174), (3, 0.1083718), (4, 0.023390446), (5, 0.0), (6, 0.0), (7, 0.035820235), (8, 0.02284322), (9, 0.0403712), (10, 0.011541335), (11, 0.010319863), (12, 0.011790952), (13, 0.0), (14, 0.0770881), (15, 0.01734209), (16, 0.024195436), (17, 0.011981061), (18, 0.02952066), (19, 0.002775732), (20, 0.015381871), (21, 0.037851065), (22, 0.01493819), (23, 0.016667154), (24, 0.016730107), (25, 0.0), (26, 0.08717003), (27, 0.024586525), (28, 0.0059973733), (29, 0.016020594), (30, 0.012602329), (31, 0.011102149), (32, 0.0), (33, 0.020326976), (34, 0.04692241), (35, 0.012366842), (36, 0.061682064), (37, 0.002117056), (38, 0.015397036), (39, 0.003181843), (40, 0.0), (41, 0.0040365355), (42, 0.013023352), (43, 0.06450635), (44, 0.042030945), (45, 0.012059403), (46, 0.008196373), (47, 0.016456507), (48, 0.01817279), (49, 0.009498764), (50, 0.0033322682), (51, 0.028735735), (52, 0.03082693), (53, 0.013